
## Introduction

The aim of this assignment is to give you the experience of using real public data and create your own report. The raw data are available in different formats and structures and often contain undesired formatting errors. Statistical data are continuously updated and may even provide API's to stream data instead of taking snapshots. We will be using snapshots in this assignment but to streamline the analysis around possible future snapshots you'll be implementing a 'data-specific' Python class with appropriate methods for data manipulation and visualisation.

**Note:** please direct your questions on this assignment to **r.monajemi@lumc.nl**

## Data: Emissions to air by the Dutch economy

In this assignment we will be using *Emissions to air by the Dutch economy* [CBS StatLine](https://opendata.cbs.nl/portal.html?_la=en&_catalog=CBS&tableId=83300ENG&_theme=1129) dataset. There you'll find the latest release of the dataset, however and older version will be provided via brightspace. Note that the results that shown below are produced using the old datasets: the metadata (`83300ENG_metadata.csv`) and the raw data (`83300ENG_UntypedDataSet_29032023_184145.csv`). 




## Assignment

#### 1) Class GWP [4p].

- Create the class `GWP` (Global Warming Potential) which is initialised with raw data along with the meta information.
- Create class attributes [emissions](emissions.md) and [emission_sources](emission_sources.md) from the metadata.
- Additional attributes:
    - `Date` :  of type timestamp created from the variable `Periods`
    - `Source` : holding the `Title` values of the emission source in the metadata.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
class GWP:
    """
    A class to interact with "Emissions to air by the Dutch Economy" CBS StatLine-dataset.

    Attributes
    ----------
    path : string
        file path directing to data .csv-file

    meta_path : string
        file path directing to metadata .csv-file

    data : pd.dataframe
        data table containing emissions of different types for economic sectors

    emission : pd.dataframe
        descriptions of emission types

    emission_sources : pd.dataframe
        descriptions of emitting economic sectors

    Methods
    -------
    emissionsGen():
        Generates emission table.

    emissionSrcGen():
        Generates emission_sources table.

    dateGen():
        Converts "Periods" in self.data to pandas Timestamp.

    convertToGreenhouseEq():
        Converts N20 and CH4 emissions to CO2-equivalents. Adds HFC as emission.

    plot(emission, source):
        Method to plot data by emission type and source.

    get_emission(category):
        Returns list containing emission types for given emission category.
    
    summary(date):
        Provides 1) general information about dataset or 2) emission summary stats for specified period.
    
    topEmitters(emission, userYear):
        Provides plot listing top 20 emitting sectors for given greenhouse gas in given year.

    """
    def __init__(self, path, meta_path):
        self.path = path
        self.meta_path = meta_path
        self.data = pd.read_csv(path, sep=";")

        self.emissionsGen()
        self.emissionSrcGen()
        self.dateGen()
        # self.convertToGreenhouseEq()

    def emissionsGen(self):
        """
        Generates emission table.
        """
        df = pd.read_csv(self.meta_path, sep=";", skiprows=4)
        df = df[df['Type'] == 'Topic']
        df.set_index(["Key"], inplace = True, drop = True)
        df = df[['Title', 'Description', 'Unit']]
        self.emission = df

  
    def emissionSrcGen(self):
        """
        Generates emission_sources table.
        """
        # "There may exist more sophisticated solutions to fetch arbitrary number of DataFrames from an untidy DataFrame."
        df = pd.read_csv(self.meta_path, sep=";", skiprows=27, nrows=59)
        df = df.rename(columns={"Key":"DutchEconomy"})

        self.source = df[['Title', 'DutchEconomy']] 
        self.source = self.source.rename(columns={"Title":"Source"})
        self.data = self.data.merge(self.source) 

        df.set_index(["DutchEconomy"], inplace = True, drop = True)
        self.emission_sources = df

    def dateGen(self):
        """
        Converts "Periods" in self.data to pandas Timestamp.
        """
        def periodToTimestamp(periodstring):
            return pd.Timestamp(int(periodstring[0:4]), 1, 1, 12)
        
        df = self.data
        df["Date"] = df["Periods"].apply(periodToTimestamp)


    def convertToGreenhouseEq(self):
        """
        Converts N20 and CH4 emissions to CO2-equivalents. Adds HFC as emission.
        """
        if 'N2O_4_IN_CO2_EQ' in self.data: # check if conversions already performed
            print("Emissions already converted to greenhouse gas equivalent.")
        else:
            self.data['N2O_4'] = self.data['N2O_4'] * 298
            self.data['CH4_5'] = self.data['CH4_5'] * 25
            self.data = self.data.rename(columns={'N2O_4': 'N2O_4_IN_CO2_EQ', 'CH4_5': 'CH4_5_IN_CO2_EQ'})

            self.data['HFC'] = self.data['GreenhouseGasEquivalents_6'] - (self.data['TotalCO2_1'] + self.data['N2O_4_IN_CO2_EQ'] + self.data['CH4_5_IN_CO2_EQ'])

    def plot(self, emission, source):
        """
        Method to plot data by emission type and source.
        
        Parameters
        ----------
        emission : list of strings
            contains variable names (e.g. "MVOC_13") to be plotted
        source : list of strings
            contains strings which pattern match emission_sources and retrieve desired sectors
        
        Returns
        -------
        seaborn plot
        """
        def check_pattern(row): # function to select Sources based on (partial) string match
            return any(row.astype(str).str.contains(pattern, case=False).any() for pattern in source)

        df = self.emission_sources.reset_index()
        df = df[df.apply(check_pattern, axis=1)]
        
        filteredData = self.data[self.data['Source'].isin(df['Title'])]
        filteredData = pd.melt(filteredData, id_vars=['Date', 'Source'], value_vars=emission, var_name="Emission")

        sns.set_style("darkgrid")
        ax = sns.lineplot(data=filteredData, x="Date", y="value", hue="Source", style="Emission")
        ax.set(ylabel='Emission (MCM)')
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), facecolor="white")

        plt.show(ax)

    def get_emission(self, category):
        """
        Returns list containing emission types for given emission category.
        
        Parameters
        ----------
        category : {"ghg", "acid", "ozone", "air"}
    
        Returns
        -------
            list of emission type variable names of given category
        """
        if category == "ghg":
            return list(self.data.columns[3:9])
        elif category == "acid":
            return list(self.data.columns[10:13])
        elif category == "ozone":
            return list(self.data.columns[[13]])
        elif category == "air":
            ugly = list(self.data.columns[14:18])
            ugly.append(list(self.data.columns[[-1]])[0])
            return ugly
        else:
            raise RuntimeError('Emission category must be \'ghg\', \'acid\', \'ozone\', or \'air\'.')
        
    def summary(self,date=None):
        """
        Provides 1) general information about dataset or 2) emission summary stats for specified period.

        Parameters
        ----------
        date : 1- or 2-element tuple of years (e.g. (1999,2008) )

        Returns
        -------
            1) general information about dataset (if no date is given)
            2) emission summary stats for given yearly period, as pd.dataframe
        """
        if date:    # if date is given: return summary stats for period
            if isinstance(date, int):
               date = tuple([date])
            dataInRange = self.data.copy()
            dataInRange = dataInRange[(dataInRange.Date.dt.year >= date[0]) & (dataInRange.Date.dt.year <= date[-1])]
            dataInRange = dataInRange.select_dtypes(include=["number"]).drop('ID', axis=1)
            
            summaries = pd.DataFrame()
            summaries["Min"] = dataInRange.min()
            summaries["1st Quantile"] = dataInRange.quantile(0.25)
            summaries["Median"] = dataInRange.median()
            summaries["Mean"] = dataInRange.mean()
            summaries["3rd Quantile"] = dataInRange.quantile(0.75)
            summaries["Max"] = dataInRange.max()
            print(f"Summary Stats for period {date[0]}-{date[-1]}:")
            return(summaries)
        else:   # if no date: print general summary of dataset
            print("General information:\n")
            print("--- FILEPATHS ---")
            print(f"Data path: \t\t\t {self.path}")
            print(f"Metadata path: \t\t\t {self.meta_path} \n")
            print("--- DATA INFO---")
            print(f"Shape of df: \t\t\t {self.data.shape[0]} rows, {self.data.shape[1]} cols.")
            print(f"Number of emission sources: \t {len(self.emission_sources)}")
            print(f"Number of emission variables: \t {len(self.emission)}")
            print(f"Data from: \t\t\t {str(gwp.data[["Date"]].sort_values(by="Date").iloc[0].item())[0:10]}")
            print(f"To: \t\t\t\t {str(gwp.data[["Date"]].sort_values(by="Date").iloc[-1].item())[0:10]}")

    def topEmitters(self, emission=0, userYear=0):
        """
        Provides plot listing top 20 emitting sectors for given greenhouse gas in given year.

        Parameters
        ----------
        userYear : int
        emission : emission variable (e.g. "MVOC_13") to be plotted (str)

        Returns
        -------
        seaborn plot
        """
        df = self.data.copy()
        df = df[df.Date.dt.year == userYear]
        df = df[~df['DutchEconomy'].isin(['1050010', 'B000579', 'B000580', 'B000581', 'T001081'])]
        df = df[["DutchEconomy", emission]].sort_values(by=emission, ascending=False)
        df = df.reset_index(drop=True)
        
        df = pd.merge(self.emission_sources, df, left_index=True, right_on="DutchEconomy", how="inner").drop(['DutchEconomy', 'Description'], axis=1)
        df = df.rename(columns={emission:gwp.emission.loc[emission]["Title"]})
        df = df.sort_values(by=df.columns[1], ascending=False)
        

        som = df.iloc[20:, 1].sum()
        df = df.iloc[:20]  
        df.loc[len(df.index)] = ["rest", som]

        ax = sns.barplot(data=df, x=df.iloc[:,1], y="Title", orient="h")
        ax.set(title=f"Top emitters of {df.columns[1]} in {userYear}.")
        ax.set(xlabel=f'{df.columns[1]} Emission (MCM)')
        ax.set(ylabel="Sector")
        plt.show(ax)

In [ ]:
gwp = GWP(path="data/83300ENG_UntypedDataSet_29032023_184145.csv", meta_path='data/83300ENG_metadata.csv')

In [ ]:
gwp.data.head()

In [ ]:
# gwp = GWP(path="data/83300ENG_UntypedDataSet_29032023_184145.csv", meta_path='data/83300ENG_metadata.csv')

In [ ]:
gwp.emission.head(10)

In [ ]:
gwp.emission_sources.head()

In [ ]:
gwp.data.dtypes

#### 2) Greenhouse equivalents [1p].

The [greenhouse equivalents](https://www.cbs.nl/en-gb/news/2020/19/uitstoot-broeikasgassen-3-procent-lager-in-2019/co2-equivalents) is a measure to relate othe other gases to CO2.

- Convert `N2O_4` and `CH4_5` in `GWP.data` to CO2 equivalents.
- Add a new variable `HFC` to the GWP.data which contains the fluorine (chlorine) gases CO2 equivalents. HFC stands for hydrofluorocarbons (see [Fluorinated gases](https://en.wikipedia.org/wiki/Fluorinated_gases)). The emission has been added to the total greenhouse gas equivalent but not specified as separate variable. See also *greenhouse gas equivalents* description in the metadata.

In [ ]:
gwp.convertToGreenhouseEq() # does N20 and CH4 conversions + adds HFC variable
gwp.data.head()


#### 3) Implement a plot method that produces the figure below. [3p]

**Synopsis:** &nbsp; &nbsp;**<tt>GWP.plot(emission, source)</tt>**

    - emission: is a list of emissions to be drawn
    - source: list of Dutch economy sector id's or a pattern

In [ ]:
gwp.plot(emission=['CO_12', 'NMVOC_13'],source=['education', 'government','320000' ])


#### 4) Emission categories [1p]

There are 4 categories of emissions:
   - Greenhouse gases (climate change) {TotalCO2_1, CO2ExclBiomass_2, CO2Biomass_3, N2O_4, CH4_5, GreenhouseGasEquivalents_6}
   - Acidification {NOx_7, SO2_8, NH3_9, AcidificationEquivalents_10}
   - Ozone layer depletion {CFK12Equivalents_11}
   - Other air pollution {CO_12, NMVOC_13, PM10_14, HFC}

Implement the method `get_emission` with the following specification:

**Synopsis:** &nbsp; &nbsp;**<tt>GWP.get_emission(emission_category)</tt>**

   - emission_category : {ghg, acid, ozone, air}
   - return value : list of variable names of the given category.


In [ ]:
# gwp.get_emission('acid')
gwp.plot(emission=gwp.get_emission('air'),source=['transport']);

#### 5) Summary [1p]

Implement the `summary` method which gives `general information` about the data such as:
- data and meta file paths
- shape of the data
- number of emission sources.
- number of emission variables
- first and latest date


**Synopsis:** &nbsp; &nbsp;**<tt>GWP.summary(date)</tt>**
   - `date`     : tuple of dates (min,[max]).

If `date` is given then print a DataFrame of descriptive statistics with columns {min, 1st quartile, 2nd quartile, mean 3rd quartile, max} and emission as row indices. In this case do not output the `general information`. Note, for the `date` method argument, in case 'max' is omitted then the summary is given for a single year.


In [ ]:
# general information
gwp.summary()

# summary statistics for 1 year
gwp.summary((1999))

# summary statistics for multiyear period
gwp.summary((1999,2005))

#### 6) Bonus point

Add a new plot method to the class which gives a different kind of information than the one given in GWP.plot.

In [ ]:
gwp.topEmitters.__doc__.split('\n')

In [ ]:
gwp.topEmitters(userYear = 2020, emission = "NOx_7")